# Summary

Grab data from AnnData object and use it to create a graph pkl

## Background

Data can be represented in a more complete way by utilizing edge features in graph attention networks and modifying the architecture based on empirical results from 2021 

## Objective

Use self-supervised learning to learn graphical representations of data and harness edge features in improving performance of predictive tasks

## Methods

GAT, edge features, self-supervised learning, representation learning, healthcare application, single-cell transcriptomic data

- use batch labels from dataset, train GAT to get edge coefficients from preds of those labels, use this for "batch effect correction" in the model, either by penalizing reliance on these edge features, or controlling for them in the final model 
- 

In [4]:
%matplotlib inline
%load_ext memory_profiler

import os
import time
import datetime
import sys
import pickle
import scanpy as sc
from scipy import sparse
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd

sc.settings.verbosity=2

In [5]:
import glob
dfp = '/home/ngr4/project/scnd/data'
glob.glob(os.path.join(dfp, '*h5ad'))

['/home/ngr4/project/scnd/data/mouse_220805.h5ad',
 '/home/ngr4/project/scnd/data/mouse_210726.h5ad']

In [6]:

import scanpy as sc
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
import random
from scipy import sparse
import pickle

import torch
import torch.nn as nn
import torch.functional as F

from torch_geometric.data import Data, ClusterData, ClusterLoader


# settings
plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['text.usetex']=False
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
sc.set_figure_params(dpi=300,dpi_save=600,
                     frameon=False,
                     fontsize=9)
plt.rcParams['savefig.dpi']=600
sc.settings.verbosity=2
sc._settings.ScanpyConfig.n_jobs=-1
sns.set_style("ticks")

# Data

## check and modify

From Leon:
correct genotype

·   5wk WT= 7202, 72921, 72922

·   5wk SCA1= 7294, 72931, 72932

·   12wk wild-type: 22018, 2061, 2062

·   12wk SCA1: 22019, 2063, 2065

·   18wk WT: 6569, 65701, 65702

·   18wk SCA1: 6571, 65731, 65732

·   24wk wild-type: 1974, 2020, 20202

·   24wk SCA1: 1589, 2021, 20212

·   30wk WT: 5812, #5822, 58232

·   30wk SCA1: #58231, 58241, 58242


58231 and 5822 have been assigned WT and SCA1 originally, respectively

In [7]:
pkl_out = '/home/ngr4/project/scnd/data/processed/mouse_220808_model_data.pkl'
adata_out = '/home/ngr4/project/scnd/data/processed/mouse_220808.h5ad'
adata_in = '/home/ngr4/project/scnd/data/mouse_220805.h5ad'

slim_pkl = '/home/ngr4/project/scnd/data/processed/mouse_220808_model_data_slim.pkl'

# load data
tic = time.time()
with open(slim_pkl, 'rb') as f:
    data = pickle.load(f)
    f.close()
print('data loaded in {:.0f}-s'.format(time.time() - tic))
print('keys:', data.keys())

data loaded in 54-s
keys: dict_keys(['pg_data', 'metadata'])


In [8]:
# modify y label
metadata = data['metadata']
pg_data = data['pg_data']
del data
target_colname = 'y_genotype_crct'
y_train = torch.tensor(metadata.loc[pg_data['train'].y, target_colname].to_numpy(dtype=np.float32))

In [10]:
# # brief chk
# # original
# dt = data['metadata'].loc[:, ['batch', 'genotype_crct', 'timepoint']].drop_duplicates().sort_values(by=['timepoint', 'genotype_crct'])
# dt.groupby(['timepoint', 'genotype_crct'])['batch'].apply(list)

In [11]:
pg_data['train'].target = y_train

In [12]:

## minibatching scheme
def minibatcher(d, batch_size=None):
    if batch_size is None:
        batch_size = int((np.sqrt(d.x.shape[0]))/32)
    cd = ClusterData(d, num_parts=int(np.sqrt(d.x.shape[0])))
    return ClusterLoader(cd, batch_size=batch_size, shuffle=True)

# initialize minibatcher
    
            
    
            
    

In [13]:
int((np.sqrt(pg_data['train'].x.shape[0]))/32)

14

In [16]:
train_idx = {i:k for i, k in enumerate(pg_data['train'].y)}
del pg_data['train'].y
pg_data['train'].idx = torch.arange(pg_data['train'].x.shape[0])

AttributeError: 'Data' object has no attribute 'y'

In [17]:
dataloader_train = minibatcher(pg_data['train'])

Computing METIS partitioning...
Done!


In [19]:
batch.target

tensor([1., 1., 1.,  ..., 1., 0., 0.])

# Induction

Sample 1/3 of the data randomly, grab labels

In [ ]:
# sample the data 
idx_train, idx_test = train_test_split(adata.obs.index, train_size=0.33)
%memit tdata = sc.AnnData(X=adata[adata.obs.index.isin(idx_train),:].X, obs=adata[adata.obs.index.isin(idx_train),:].obs)
temp = adata.obs.index[adata.obs.index.isin(idx_test)].to_list()
idx_val, idx_test = train_test_split(temp, train_size=0.1)
val = sc.AnnData(X=adata[adata.obs.index.isin(idx_val),:].X, obs=adata[adata.obs.index.isin(idx_val),:].obs)
temp = adata.obs.index[adata.obs.index.isin(idx_test)].to_list()
idx_test, _ = train_test_split(temp, train_size=0.11)
test = sc.AnnData(X=adata[adata.obs.index.isin(idx_test),:].X, obs=adata[adata.obs.index.isin(idx_test),:].obs)

def graph_pp(AnnData, bbknn=True):
    sc.tl.pca(AnnData, n_comps=50)
    if bbknn:
        sc.external.pp.bbknn(AnnData)
    else:
        sc.pp.neighbors(AnnData, n_pcs=100, n_neighbors=30)
    return AnnData

# make graph
tdata = graph_pp(tdata)
val = graph_pp(val)
test = graph_pp(test)

if False:
    del adata

## Encoding 

Select tasks for prediction

1. yctype
2. ysca1
3. ygenotime (already done)
4. SCA1_5/12/18/24/30wk


In [ ]:
# encode ctype 
ctype_encoder = {v:i for i,v in enumerate(tdata.obs['ctype'].unique())}
tdata.obs['yctype'] = tdata.obs['ctype'].map(ctype_encoder)
val.obs['yctype'] = val.obs['ctype'].map(ctype_encoder)
test.obs['yctype'] = test.obs['ctype'].map(ctype_encoder)

# encode WT/SCA1 blended across time
genotype_encoder = {'WT':0, 'SCA1':1}
tdata.obs['ysca1'] = tdata.obs['genotype'].map(genotype_encoder).astype(int)
val.obs['ysca1'] = val.obs['genotype'].map(genotype_encoder).astype(int)
test.obs['ysca1'] = test.obs['genotype'].map(genotype_encoder).astype(int)

# encode multi-label
tdata.obs['genotype_timepoint'] = tdata.obs['genotype'].astype(str) + tdata.obs['timepoint'].astype(str).apply(lambda x: '_{}'.format(x))
val.obs['genotype_timepoint'] = val.obs['genotype'].astype(str) + val.obs['timepoint'].astype(str).apply(lambda x: '_{}'.format(x))
test.obs['genotype_timepoint'] = test.obs['genotype'].astype(str) + test.obs['timepoint'].astype(str).apply(lambda x: '_{}'.format(x))

gt_encoder = {v:i for i,v in enumerate(tdata.obs['genotype_timepoint'].unique())}
tdata.obs['ygenotime'] = tdata.obs['genotype_timepoint'].map(gt_encoder)
val.obs['ygenotime'] = val.obs['genotype_timepoint'].map(gt_encoder)
test.obs['ygenotime'] = test.obs['genotype_timepoint'].map(gt_encoder)

# encode distinguishability of SCA1 at specific timepoints 
verbose = False
tdata.obs['SCA1_5wk'] = (tdata.obs['genotype_timepoint']=='SCA1_5wk').astype(int)
tdata.obs['SCA1_12wk'] = (tdata.obs['genotype_timepoint']=='SCA1_12wk').astype(int)
tdata.obs['SCA1_18wk'] = (tdata.obs['genotype_timepoint']=='SCA1_18wk').astype(int)
tdata.obs['SCA1_24wk'] = (tdata.obs['genotype_timepoint']=='SCA1_24wk').astype(int)
tdata.obs['SCA1_30wk'] = (tdata.obs['genotype_timepoint']=='SCA1_30wk').astype(int)
val.obs['SCA1_5wk'] = (val.obs['genotype_timepoint']=='SCA1_5wk').astype(int)
val.obs['SCA1_12wk'] = (val.obs['genotype_timepoint']=='SCA1_12wk').astype(int)
val.obs['SCA1_18wk'] = (val.obs['genotype_timepoint']=='SCA1_18wk').astype(int)
val.obs['SCA1_24wk'] = (val.obs['genotype_timepoint']=='SCA1_24wk').astype(int)
val.obs['SCA1_30wk'] = (val.obs['genotype_timepoint']=='SCA1_30wk').astype(int)
test.obs['SCA1_5wk'] = (test.obs['genotype_timepoint']=='SCA1_5wk').astype(int)
test.obs['SCA1_12wk'] = (test.obs['genotype_timepoint']=='SCA1_12wk').astype(int)
test.obs['SCA1_18wk'] = (test.obs['genotype_timepoint']=='SCA1_18wk').astype(int)
test.obs['SCA1_24wk'] = (test.obs['genotype_timepoint']=='SCA1_24wk').astype(int)
test.obs['SCA1_30wk'] = (test.obs['genotype_timepoint']=='SCA1_30wk').astype(int)

verbose = False
if verbose:
    # check encoding 
    print(tdata.obs['genotype_timepoint'].value_counts())
    for i in ['SCA1_5wk', 'SCA1_12wk','SCA1_18wk','SCA1_24wk','SCA1_30wk']:
        print(tdata.obs[i].sum())
        
    print(test.obs['genotype_timepoint'].value_counts())
    for i in ['SCA1_5wk', 'SCA1_12wk','SCA1_18wk','SCA1_24wk','SCA1_30wk']:
        print(test.obs[i].sum())



In [ ]:
# create dictionary
def dictthat(AnnData, gene_ranger=True):
    """Prep dictionary for export.
    
    If gene_ranger, divide by zero can occur for 
    non-expressing genes. Thus, will floor those
    to 0.
    
    NOTE: customization re:y to predict is highly
    dependent on user input. ERGO, modify this 
    
    Arguments:
        AnnData (sc.AnnData): with graph stuff
        
    Returns:
        dict
    """
    if gene_ranger:
        # each gene in [0,1], divide by zeros to 0
        minimum = AnnData.X.min(axis=0)
        maximum = AnnData.X.max(axis=0)
        num = AnnData.X - minimum.todense()
        denom =  (maximum - minimum).todense()
        xhat = np.divide(num, denom, out=np.zeros_like(num), where=denom!=0) 
    else:
        # matrix in [0,1]
        xhat = (AnnData.X - AnnData.X.min()) / (AnnData.X.max() - AnnData.X.min())
        
    

    gdata = {'X':xhat,
             'adj':AnnData.uns['neighbors']['connectivities']+sparse.diags([1]*AnnData.shape[0], format='csr'),
             'feature_names':AnnData.var_names.to_list()}
    gdata['cell_id'] = AnnData.obs.index.to_list()
    for col in AnnData.obs.columns:
        gdata[col] = AnnData.obs[col].to_list()
    
    return gdata

gdata_train = dictthat(tdata)
gdata_val = dictthat(val)
gdata_test  = dictthat(test)



In [ ]:
# export
def pklthat(gdata, fname, fpath=pdfp): 
    with open(os.path.join(fpath,fname),'wb') as f :
        pickle.dump(gdata, f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()

pklthat(gdata_train, 'scnd_train_200528.pkl')
pklthat(gdata_val, 'scnd_val_200528.pkl')
pklthat(gdata_test, 'scnd_test_200528.pkl')

# clean
if True:
    del tdata, test, gdata_train, gdata_test

## modify pkl

Add batch encoding

In [ ]:
def loadpkl(filename):
    with open (filename, 'rb') as f:
        temp = pickle.load(f)
        f.close()
    return temp

def add_batch(filename, date='200529'):
    gdata = loadpkl(filename)
    batch_encoder = {v:i for i,v in enumerate(np.unique(gdata['batch']))}
    gdata['ybatch'] = list(map(batch_encoder.get, gdata['batch']))
    pklthat(gdata, '{}_{}.pkl'.format(os.path.split(filename)[1].split('_20')[0], date))
    del gdata
    print('Batch added and pkl saved:\n  {}_{}.pkl'.format(os.path.split(filename)[1].split('_20')[0], date))

In [ ]:
add_batch(os.path.join(pdfp,'scnd_train_200528.pkl'))
add_batch(os.path.join(pdfp,'scnd_val_200528.pkl'))
add_batch(os.path.join(pdfp,'scnd_test_200528.pkl'))

In [ ]:
# test
gdata = loadpkl(os.path.join(pdfp,'scnd_train_200529.pkl'))